In [2]:
import csv
import pandas as pd
# Increase the maximum field size limit
csv.field_size_limit(10**8)

100000000

In [3]:
def split_list_into_parts(input_list, part_size):
    return [input_list[i:i + part_size] for i in range(0, len(input_list), part_size)]


In [ ]:
# Specify the path to your CSV file
file_path = '../data/20181024_d1_0830_0900.csv'

# Open the CSV file
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    
    i = 0
    # Read each row one by one
    for line_number, row in enumerate(csv_reader):
        # Process individual lines as needed
        if (i == 0):
            # create the column names for the fact and dimensional table
            columns = row[0]
            dimension_columns = [columns.split(";")[0]] + columns.split(";")[4:]
            
            # create fact and dimensional table
            fact_df = pd.DataFrame(columns=columns.split(";")[:4])
            dimension_df = pd.DataFrame(columns=dimension_columns)          
        else:
            data = [elt  for elt in row[0].split(";") if elt != " "]
            track_id = data[0]
            
            fact_df.loc[len(fact_df)] = data[:4]
            
            # Split the list into parts containing 6 elements each
            split_parts = split_list_into_parts(data[4:], 6)

            for dimensions in split_parts:
                dimensions.insert(0, track_id)
                dimension_df.loc[len(dimension_df)] = dimensions
        i+=1        

In [7]:
print(fact_df.shape)
print(dimension_df.shape)

(0, 4)
(0, 818)


In [8]:
fact_df

,922,Car,35.00,23.335533
